# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
from __future__ import print_function
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt



%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [19]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.345362
sanity check: 2.302585


$$f(x, W) = - \log{(truescore(\frac{X.dot(W)}{sum(X.dot(W))}))} $$

In [5]:
X_dev.shape, W.shape

((500, 3073), (3073, 10))

In [14]:
reg = 5e1
dW= np.zeros(W.shape)
scores = np.zeros((X_dev.shape[0], W.shape[1]))
for i in range(0, X_dev.shape[0]):
    for j in range(0, W.shape[1]):
        dot = np.sum(X_dev[i, :]*(W[:, j])) #1 (1)
        scores[i, j] = dot  #(1)
    scores[i, :] -= np.max(scores[i, :]) #avoiding numerical stabilization
    scores_exp = np.exp(scores[i, :]) #2 (1,10)
    num = scores_exp #3 #numerator (1,10)
    scores_sum = scores_exp.sum() #3 (1)
    den = scores_sum #3 (1)
    invden = 1 / den #3.5 #denumerator > (1) 
    probs = num * invden #4 #sigmoid is done (1, 10) #broadcast
    tclass_prob = probs[y_dev[i]] #5 (1)
    tclass_lprob = np.log(tclass_prob) #6 (1)
    loss += -(tclass_lprob) #7 (1)
    
    #derivative with chain rule
    dloss = 1
    dtclass_lprob = -1*dloss #7 # shape > 1
    dtclass_prob = 1/tclass_prob*dtclass_lprob #6  # shape > 1
    
    dnum = invden*dtclass_prob #(1)
    dinvden = num*dtclass_prob #(1,10) #broadcast
    
    dden = (-1 / den**2)*dinvden #(1,10) #broadcast
    dscores_sum = 1*dden #(1,10)
    dscores_exp = 1*dnum #(1)
    dscores_exp += dscores_exp*dscores_sum #(1,10)broadcast
    
    dscores = np.exp(scores[i, :])*dscores_exp #(1,10)
    ddot = dscores #(1,10)
    dW += X_dev[i, :].reshape(-1,1).dot(dscores.reshape(1,-1)) # dscores > (1,10) dW > 3073x10
    
#     dscores_exp = inv_scores_sum*dnormalized_scores #4 # 1 number
#     dinv_scores_sum = scores_exp*dnormalized_scores #4 # (1,10), true class score affect all of the classes?
    
#     dscores_sum = (-1/scores_sum**2)*dinv_scores_sum #4
#     dscores_exp += dscores_sum.sum() #3
#     dscores_i = np.exp(scores[i, :])*dscores_exp #2
#     ddot = scores[i, :] * dscores_i #1
#     dW += X_dev[i, :].reshape(W.shape[0],-1) * ddot.reshape(1, -1) 
#     #ddot*X_dev[i, :]
    
#     #ddot = ((1 - scores[i, y_dev[i]])*scores[i, y_dev[i]])*d_loss
#     #dW[:, y_dev[i]] += X_dev[i, :] * ddot
loss = loss / X_dev.shape[0]
loss += reg*np.sum(W*W)
dW = dW / X_dev.shape[0]
dW += reg*2*W
print(loss)
print(dW)

2.4055826558478217
[[ -4.19670602  28.40761452  -3.45568219 ... -18.12122782 -24.14348344
  -53.10404278]
 [ -4.87149289  31.49540657  -7.80340968 ... -21.23371776 -25.29159951
  -53.47359996]
 [ -4.00125805  34.69208264 -10.13795198 ... -24.48288757 -26.8513872
  -56.8727704 ]
 ...
 [ -0.83919133   4.08942176  -9.78945364 ...   5.23860806 -22.46878681
  -44.71419916]
 [  2.67989202   7.47984659 -10.22568158 ...   2.98665111 -23.98608068
  -49.3042809 ]
 [ -1.40397396  -1.46970459  -1.36763732 ...  -1.39557877  -1.44232003
   -1.71540541]]


In [12]:
X_dev[1, :].shape

(3073,)

## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *Initially all weights are close to 0, so scores. If we exponentiate the scores, we get 1 for each class and example. Then we normalize them, they are going to be 1/number of classes. And loss is -log(1/number of classes) which in this case is -log(0.1)*


In [21]:
np.random.seed(18)

In [22]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: -0.189089 analytic: 12.588178, relative error: 1.000000e+00
numerical: -0.444305 analytic: -8.202051, relative error: 8.972273e-01
numerical: -4.457800 analytic: 17.383809, relative error: 1.000000e+00
numerical: -5.255162 analytic: 8.396065, relative error: 1.000000e+00
numerical: -0.342034 analytic: 4.335069, relative error: 1.000000e+00
numerical: 5.286925 analytic: -12.207131, relative error: 1.000000e+00
numerical: -1.677300 analytic: -16.216522, relative error: 8.125274e-01
numerical: -1.545366 analytic: 15.928681, relative error: 1.000000e+00
numerical: 2.719497 analytic: -16.419000, relative error: 1.000000e+00
numerical: 2.145486 analytic: -11.246855, relative error: 1.000000e+00
numerical: 1.592355 analytic: -2.617703, relative error: 1.000000e+00
numerical: -0.204121 analytic: -4.956848, relative error: 9.208981e-01
numerical: 1.639391 analytic: 3.441682, relative error: 3.547068e-01
numerical: -0.904669 analytic: 11.892206, relative error: 1.000000e+00
numerical:

In [13]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.312187e+00 computed in 0.124292s
vectorized loss: 2.312187e+00 computed in 0.001800s
Loss difference: 0.000000
Gradient difference: 0.000000


In [21]:
#vectorized loss
N = X_dev.shape[0] #number of examples
scores_dev = X_dev.dot(W) #1
scores_dev -= np.max(scores_dev, axis = 1).reshape(N, 1) #for numerical stability
exp_dev = np.exp(scores_dev) #2
sum_scores_dev = exp_dev.sum(axis=1).reshape(N, 1) #3
inv_sum_scores_dev = 1 /sum_scores_dev #4
probs = exp_dev *inv_sum_scores_dev #5
true_class_probs = probs[range(0,N), y_dev] #6
losses = -np.log(true_class_probs) #7
total_loss = np.sum(losses) / N # it is average over all examples
print(total_loss, "without regularization")

2.3121870572013106 without regularization


In [22]:
print(true_class_probs.shape)

(500,)


In [24]:
X_dev.shape, W.shape, scores_dev.shape

((500, 3073), (3073, 10), (500, 10))

In [26]:
dlosses = - 1 / true_class_probs #7
d_true_class_probs = 1*dlosses #6 
dexp_dev = inv_sum_scores_dev*d_true_class_probs #5
dinv_sum_scores_dev = exp_dev.sum(axis=1)*d_true_class_probs #5
d_sum_scores_dev = (-1 / sum_scores_dev**2)*dinv_sum_scores_dev #4
dexp_dev_sum = 1*d_sum_scores_dev #3
dscores_dev = np.exp(scores_dev)*dexp_dev #2
dW = X_dev.T.dot(dscores_dev)
print(dW)

ValueError: operands could not be broadcast together with shapes (500,10) (500,500) 

In [94]:
probs.shape

(500, 10)

In [97]:
dloss_dev = -1/ probs[range(0,X_dev.shape[0]), y_dev]
dloss_dev

array([-10.24344591, -10.0693846 ,  -7.21488314, -10.05059846,
        -9.6885657 ,  -9.09567875, -13.19385224, -10.75194813,
        -7.59678406,  -7.09782924, -10.38920684,  -8.44498651,
       -11.71486451, -17.86464526, -12.39219935,  -4.26318896,
        -5.62145284, -10.51795405,  -7.42975185, -18.20841223,
       -12.16054727, -10.52370702, -13.54912635, -15.2144803 ,
        -9.81524152, -11.78209259, -15.52690029, -15.75998632,
        -9.35495608, -10.44093938, -11.25489778, -14.70446588,
       -12.59559697, -11.11331942,  -9.31502842, -25.0585745 ,
       -12.95305913,  -8.25955757,  -4.94753002,  -6.81677992,
       -13.36469739,  -8.49564372, -11.4601513 ,  -7.23243183,
        -7.72567799,  -8.94667179, -18.91175583, -12.13044704,
        -7.93887352, -15.43127958, -12.04393639, -13.64336076,
       -12.42898409,  -8.77216919, -13.15067272, -13.14381448,
        -8.40257197,  -7.11853199,  -7.07776234, -16.09172783,
        -9.91562211, -12.2269092 ,  -5.05994696, -10.10

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
# Your code
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question** - *True or False*

It's possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

*Your answer*:

*Your explanation*:

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])